### 手动编写编译器加速
Python默认编译器在编译GP时实际上速度较慢，因此我们可以考虑自行实现一个编译器来加速GP运算。

In [19]:
from deap.gp import PrimitiveTree, Primitive, Terminal
import time

import numpy as np
from deap import base, creator, tools, gp

def quick_evaluate(expr: PrimitiveTree, pset, data, prefix='ARG'):
    result = None
    stack = []
    for node in expr:
        stack.append((node, []))
        while len(stack[-1][1]) == stack[-1][0].arity:
            prim, args = stack.pop()
            if isinstance(prim, Primitive):
                result = pset.context[prim.name](*args)
            elif isinstance(prim, Terminal):
                if prefix in prim.name:
                    result = data[:, int(prim.name.replace(prefix, ''))]
                else:
                    result = prim.value
            else:
                raise Exception
            if len(stack) == 0:
                break  # If stack is empty, all nodes should have been seen
            stack[-1][1].append(result)
    return result

# 符号回归
def evalSymbReg(individual, pset):
    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100).reshape(-1,1)
    
    # 评估生成的函数并计算MSE
    mse = np.mean((quick_evaluate(individual,pset,x) - x**2)**2)
    
    return (mse,)

# 创建个体和适应度函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [20]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(np.add, 2)
pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.negative, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

In [21]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
start=time.time()
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.8, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)
end=time.time()
print('time:',end-start)
print(str(hof[0]))

   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max   	min    	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	2801.71	0  	153712	1849.78	300   	8775.53	4.01333	0  	7  	2  	300   	1.66928
1  	241   	2154.6 	1  	6781.67	1849.78	241   	617.2  	4.44   	1  	13 	2  	241   	2.1291 
2  	250   	3221.44	2  	153712 	1849.78	250   	12044.1	4.97667	2  	16 	2  	250   	2.45685
3  	245   	5845.69	3  	162303 	1849.78	245   	23177.9	5.47333	3  	17 	2  	245   	2.54741
4  	248   	8351.8 	4  	612303 	1849.78	248   	42618.1	5.89333	4  	22 	2  	248   	3.12655
5  	250   	8533.02	5  	166663 	1849.78	250   	29977.2	6.16667	5  	24 	2  	250   	3.81648
6  	264   	7435.01	6  	158106 	1849.78	264   	27133.8	5.48667	6  	24 	2  	264   	3.02707
7  	259   	50370.8	7  	1.24977e+07	1849.78	259   	721154 	5.38333	7  	24 	2 